<a href="https://colab.research.google.com/github/mcbaccam/Anal-tica-de-Negocio/blob/main/RetoN%C2%B02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Arboles de decisión**
0. Cargar las librerías de referencia para el modelo

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt #Graficador de cosas
from sklearn.tree import DecisionTreeClassifier

#Para no tener que cargar el archivo
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Se procede con la carga de los datos del archivo SolicitantesCrédito

In [ ]:
nxl= '/content/2. BD1_Travel_Data_int.xlsx'
XDB = pd.read_excel(nxl, sheet_name = 0)
XDB.head(100)

,College,CreditCard,FoodSpend,Income,TravelSpend
0,1,1,5472.43,49150,827.40
1,0,1,9130.73,47806,863.55
2,0,1,4450.67,46050,1180.05
3,0,1,5020.72,42600,755.70
4,0,1,3408.11,52092,851.40
...,...,...,...,...,...
95,0,0,2019.98,65982,4338.75
96,0,0,7046.11,53986,1851.60
97,0,0,5180.13,53986,1646.70
98,0,0,3266.44,63962,1641.60


In [ ]:
#Se procede con la selección de las variables de trabajo
XD=np.array(XDB[['College','FoodSpend','Income','TravelSpend']]) #Datos Entrada
print(XD)
yd=np.array(XDB[['CreditCard']]) #Este es el dato que quiero que el modelo pronostique
#print(yd)

[[1.00000e+00 5.47243e+03 4.91500e+04 8.27400e+02]
 [0.00000e+00 9.13073e+03 4.78060e+04 8.63550e+02]
 [0.00000e+00 4.45067e+03 4.60500e+04 1.18005e+03]
 ...
 [0.00000e+00 3.58586e+03 4.22500e+04 2.25195e+03]
 [1.00000e+00 1.83395e+03 6.02480e+04 2.00025e+03]
 [0.00000e+00 6.20597e+03 5.85200e+04 3.66750e+03]]


2. Creacion del modelo - Decision Tree

In [ ]:

# Encode target variable
mar=DecisionTreeClassifier(criterion='gini', max_depth = 4) #4 Niveles o variables de entrada
mar.fit(XD,yd)
ydp=mar.predict(XD)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(yd,ydp)
print(cm)

#Evaluamos el modelo
VN=cm[0][0]; FP=cm[0][1]; FN=cm[1][0]; VP=cm[1][1]

Ex=(VN+VP)/(VN+VP+FN+FP) #Que porcentaje de los datos pronostico correctamente
print("La exactitud alcanzada por el modelo es:",Ex)
Se=VP/(FN+VP) #Como le fue con los positivos
print("La sensibilidad alcanzada por el modelo fue:",Se)
Es=VN/(VN+FP) #Como le fue con los negativos
print("La especificidad alcanzada por el modelo fue:",Es)


[[334   9]
 [ 42 115]]
La exactitud alcanzada por el modelo es: 0.898
La sensibilidad alcanzada por el modelo fue: 0.732484076433121
La especificidad alcanzada por el modelo fue: 0.9737609329446064


3. ¿ Y dónde está el árbol?

In [ ]:
#Estas son las librerias para graficar árbol
from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data

VS = ['College','FoodSpend','Income','TravelSpend'] #Son los títulos para los nodos
dot_graph=export_graphviz(mar,feature_names=VS)
graph=graph_from_dot_data(dot_graph)
graph.write_png('arbol.png')

True

4. Se procede con el pronostico de los nuevos solicitantes

In [ ]:
XDB2=pd.read_excel(nxl, sheet_name=1)
XDB2 = XDB2.dropna() #Elimina las celdas vacías o sin datos
XDB2.head(100)

XD2= np.array(XDB2[['College','FoodSpend','Income','TravelSpend']])
print(XD2)

#Pronosticamos la preaprobacion
ydp2 = mar.predict(XD2)
print(ydp2)
print( "El número de preaprobados es:" ,np.sum(ydp2))
print( "El numero total de Datos es:" , len(ydp2))
print( "El numero de PreNegados es:" , len(ydp2)-np.sum(ydp2))

[[0.00000e+00 2.89290e+03 6.59820e+04 2.15160e+03]
 [1.00000e+00 6.01766e+03 5.39860e+04 2.00250e+03]]
[0 0]
El número de preaprobados es: 0
El numero total de Datos es: 2
El numero de PreNegados es: 2


**Análisis de resultados**

De acuerdo con la base de datos sumistrada y el nodo base del arbol de decision se puede observar yd total de 500 datos, divididos en 343 Preaprobado y un total de 157 Prenegados, para predecir si un cliente utilizará su tarjeta de crédito para cubrir gastos de viaje y alimentos.
De acuerdo al pronóstico realizado, se alcanzó una exactitud de 89%, se obtuvo una sensibilidad mas baja de 73%, indicando que se estan perdiendo el 26% de los casos positivos, clasificándolos erróneamente como falsos negativos (FN). Se obtuvo una especificidad del 97%.

Aparecen 3 Nodos Puros
- Nodo 1  "[PreNeg:1,PreApr:0]" . Se debe cumplir conla regla de:
TravelSpend <= 1188.0 AND FoodSpend <= 4990.73 AND FoodSpend > 4972.55
- Nodo 2 - "[PreNeg:0,PreApr:47]". Se debe cumplir con la regla de:
TravelSpend <= 1188.0 AND FoodSpend > 4990.73
- Nodo 3 - "[PreNeg:108,PreApr:0]". Se debe cumplir la regla de:
 TravelSpend > 1567.875 AND Income > 59098.0


Con respecto al pronostico, podemos ver un total de 2 individuos sin preparobacion.  De los cuales 0 son preaprobados y un total de 2 prenegados